In [1]:
import explainability as exp
import pandas as pd

c:\Users\anton\anaconda3\envs\INV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Regression Models

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=10, n_informative=10, n_targets=1, random_state=123)
X = pd.DataFrame(X)
reg = LinearRegression()

reg.fit(X,y)

print(f"Algorithm Class: {exp.algorithm_class_score(reg)}")
print(f"Model Size Score: {exp.model_size_score(X)}")

print(f"Correlated Features Score: {exp.correlated_features_score(X)}")
print(f"Feature Importance Score: {exp.feature_importance_score(reg)}")
print(f"Shap Coefficient of variance: {exp.cv_shap_score(reg, X)}")


Algorithm Class: 3.5
Model Size Score: 5
Correlated Features Score: 1
Feature Importance Score: 5
Shap Coefficient of variance: 0.9597673377706508


### Classification Models

In [4]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

X, y = make_classification(n_samples=1000, n_features=10, n_informative=10,flip_y=0, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=5, random_state=42)
X = pd.DataFrame(X)
clf = RandomForestClassifier(random_state=123)

clf.fit(X,y)


print(f"Algorithm Class: {exp.algorithm_class_score(clf)}")
print(f"Model Size Score: {exp.model_size_score(X)}")

print(f"Correlated Features Score: {exp.correlated_features_score(X)}")
print(f"Feature Importance Score: {exp.feature_importance_score(clf)}")
print(f"Shap Coefficient of variance: {exp.cv_shap_score(clf, X)}")

Algorithm Class: 4
Model Size Score: 5
Correlated Features Score: 1
Feature Importance Score: 5
Shap Coefficient of variance: 0.6969347183053155


### Keras Models

In [5]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

X, y = make_classification(n_samples=1000, n_features=10, n_informative=10,flip_y=0, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=5, random_state=42)
X = pd.DataFrame(X)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

TFmodel = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation="softmax")
])

TFmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
TFmodel.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

print(f"Algorithm Class: {exp.algorithm_class_score(TFmodel)}")
print(f"Model Size Score: {exp.model_size_score(X)}")

print(f"Correlated Features Score: {exp.correlated_features_score(X)}")
print(f"Feature Importance Score: {exp.feature_importance_score(TFmodel)}")
print(f"Shap Coefficient of variance: {exp.cv_shap_score(TFmodel, X)}")

Epoch 1/10


c:\Users\anton\anaconda3\envs\INV\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2003 - loss: 2.5791 - val_accuracy: 0.2100 - val_loss: 2.0454
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2411 - loss: 2.2748 - val_accuracy: 0.2450 - val_loss: 1.8294
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2604 - loss: 2.1646 - val_accuracy: 0.2900 - val_loss: 1.6669
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3254 - loss: 1.8466 - val_accuracy: 0.3400 - val_loss: 1.5418
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3232 - loss: 1.7992 - val_accuracy: 0.3650 - val_loss: 1.4365
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3442 - loss: 1.7696 - val_accuracy: 0.4000 - val_loss: 1.3466
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3856 - loss: 1.5023 - val_accuracy: 0.4500 - val_loss: 1.2797
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3925 - loss: 1.5207 - val_accuracy: 0.4850 - val_loss: 1.2271
Ep

### Torch Models

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset, random_split

# Generate synthetic dataset
X, y = make_classification(
    n_samples=1000, n_features=10, n_informative=10, flip_y=0, n_redundant=0, 
    n_repeated=0, n_clusters_per_class=2, n_classes=5, random_state=42
)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_onehot = torch.tensor(encoder.fit_transform(y.reshape(-1, 1)), dtype=torch.float32)

# Create a dataset and dataloaders
dataset = TensorDataset(X, y_onehot)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define the model
class TorchModel(nn.Module):
    def __init__(self):
        super(TorchModel, self).__init__()
        self.dense1 = nn.Linear(10, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.dense2 = nn.Linear(32, 5)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.softmax(self.dense2(x))
        return x

    def predict(self, x):
        # x is numpy not tensor, return is numpy
        xx = torch.tensor(x, dtype=torch.float32).to(device)
        
        with torch.no_grad():
            probs = torch.exp(self.forward(xx))
        return probs.numpy()

# Create an instance of the model
Tmodel = TorchModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Tmodel.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    Tmodel.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = Tmodel(inputs)
        loss = criterion(outputs, labels.argmax(dim=1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / train_size
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    # Validation loop
    Tmodel.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = Tmodel(inputs)
            loss = criterion(outputs, labels.argmax(dim=1))
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels.argmax(dim=1)).sum().item()
    
    val_loss /= val_size
    accuracy = correct / val_size
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')


print(f"Algorithm Class: {exp.algorithm_class_score(Tmodel)}")
print(f"Model Size Score: {exp.model_size_score(X)}")

print(f"Correlated Features Score: {exp.correlated_features_score(X)}")
print(f"Feature Importance Score: {exp.feature_importance_score(Tmodel)}")
#print(f"Shap Coefficient of variance: {exp.cv_shap_score(Tmodel, X)}")


c:\Users\anton\anaconda3\envs\INV\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10, Loss: 1.6192
Validation Loss: 1.6016, Accuracy: 0.2350
Epoch 2/10, Loss: 1.5911
Validation Loss: 1.5799, Accuracy: 0.3450
Epoch 3/10, Loss: 1.5637
Validation Loss: 1.5609, Accuracy: 0.4050
Epoch 4/10, Loss: 1.5495
Validation Loss: 1.5412, Accuracy: 0.4100
Epoch 5/10, Loss: 1.5177
Validation Loss: 1.5221, Accuracy: 0.4300
Epoch 6/10, Loss: 1.4979
Validation Loss: 1.5030, Accuracy: 0.4500
Epoch 7/10, Loss: 1.4766
Validation Loss: 1.4873, Accuracy: 0.4600
Epoch 8/10, Loss: 1.4647
Validation Loss: 1.4718, Accuracy: 0.4700
Epoch 9/10, Loss: 1.4362
Validation Loss: 1.4576, Accuracy: 0.4700
Epoch 10/10, Loss: 1.4228
Validation Loss: 1.4448, Accuracy: 0.4650
Algorithm Class: 1
Model Size Score: 5
Correlated Features Score: 1
Feature Importance Score: None


In [17]:
import shap
background = shap.sample(X, 100)
explainer = shap.KernelExplainer(Tmodel, background)


TypeError: Unknown type passed as data object: <class 'torch.Tensor'>

In [3]:
print(f"Shap Coefficient of variance: {exp.cv_shap_score(Tmodel, X)}")

TypeError: Unknown type passed as data object: <class 'torch.Tensor'>